In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
pip freeze >> requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%matplotlib inline
DATAPATH = 'sample/stock_prices_sample.csv'
data = pd.read_csv(DATAPATH, index_col=['DATE'], parse_dates=['DATE'])
data.head(10)

In [ ]:
#clean data
data = data[data.TICKER != 'GEF']
data = data[data.TYPE != 'Intraday']

drop_cols = ['VOLUME','SPLIT_RATIO', 'EX_DIVIDEND', 'ADJ_FACTOR', 'ADJ_VOLUME', 'ADJ_CLOSE', 'ADJ_LOW', 'ADJ_HIGH', 'ADJ_OPEN', 'FREQUENCY', 'TYPE', 'FIGI']
data.drop(drop_cols, axis=1, inplace=True)
data.head()

In [ ]:
#plot closing price
plt.figure(figsize=(10,5))
plt.plot(data.CLOSE)
plt.title('Closing price of New GF Inc')
plt.ylabel('Closing price ($)')
plt.xlabel('Trading day')
plt.show()

In [ ]:
#moving average model 
#the next observation is the mean of all past observations
#moving average model uses past forecast errors in a regression
from sklearn.metrics import mean_absolute_error

def plt_moving_avg(series, window, plot_intervals=False, scale=1.96):
    rolling_mean = series.rolling(window=window).mean()
    plt.figure(figsize=(15,7))
    plt.title('Moving Avg \n Window size = {}'.format(window))
    plt.plot(rolling_mean, 'g', label='Rolling mean trend')

    #plot confidence intervals for smooth values by the previous week/month/quater data
    if plot_intervals:
        mean_abs_err = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bound = rolling_mean - (mean_abs_err + scale * deviation)
        upper_bound = rolling_mean + (mean_abs_err + scale * deviation)

        plt.plot(upper_bound, 'r--', label='Upper bound / Lower bound')
        plt.plot(lower_bound, 'r--')

    plt.plot(series[window:], label='Actual values')
    plt.legend(loc='best')
    plt.grid(True)

#smooth by previous 5 days
plt_moving_avg(data.CLOSE, 5)
#smooth by previous month
plt_moving_avg(data.CLOSE, 30)
#smooth by previous quarter
plt_moving_avg(data.CLOSE, 90, plot_intervals=True)

In [ ]:
#exponential smoothing
#assigns exponentially decreasing weights over time to the past observations
#less importance is given to observations as we move further from the present
def expo_smoothing(series, alpha):
    result = [series[0]] # first value as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1]) #y𝑡=∝x𝑡+(1−∝)y𝑡−1, 𝑡>0 and alpha is smoothing factor
    return result
  
def plot_expo_smoothing(series, alphas):
    plt.figure(figsize=(15, 7))
    for alpha in alphas:
        plt.plot(expo_smoothing(series, alpha), label="Alpha {}".format(alpha))
    plt.plot(series.values, "c", label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Exponential smoothing")
    plt.grid(True);

plot_expo_smoothing(data.CLOSE, [0.05, 0.3])

In [ ]:
#Modeling
#when we make a model for forecasting purposes in time series analysis, we require a stationary time series for better prediction.
#ADF Fuller test is a statistical test used to check for stationarity in time series. 
from statsmodels.tsa.stattools import adfuller
import statsmodels.tsa.api as smt

def tsplot(y, lags=None, figsize=(15, 7), syle='bmh'):    
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style='bmh'):
        fig = plt.figure(figsize=figsize)
        layout = (2,2)
        ts_ax = plt.subplot2grid(layout, (0,0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1,0))
        pacf_ax = plt.subplot2grid(layout, (1,1))
        y.plot(ax=ts_ax)
        p_value = adfuller(y)[1]
        ts_ax.set_title('Time series analysis\n Dickey-Fuller: p={0:.5f}'.format(p_value))
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()
        
tsplot(data.CLOSE, lags=30)

# Take the first difference to remove to make the process stationary
data_diff = data.CLOSE - data.CLOSE.shift(1)

tsplot(data_diff[1:], lags=30)

In [ ]:
#SARIMA
# - we first need to define a few parameters and a range of values for 
# other parameters to generate a list of all possible combinations of p, q, d, P, Q, D, s.
from itertools import product
from tqdm import tqdm_notebook
import statsmodels.api as sm

#Set initial values and some bounds
ps = range(0, 5)
d = 1
qs = range(0, 5)
Ps = range(0, 5)
D = 1
Qs = range(0, 5)
s = 5

#Create a list with all possible combinations of parameters
params = product(ps, qs, Ps, Qs)
params_list = list(params)
len(params_list)

# Train many SARIMA models to find the best set of params
def optimize_SARIMA(params_list, d, D, s):
    """ Return dataframe with params and corresponding AIC
        params_list - list with (p, q, P, Q) tuples
        d - integration order
        D - seasonal integration order
        s - length of season
    """    
    results = []
    best_aic = float('inf')
    
    for param in tqdm_notebook(params_list):
        try: 
            model = sm.tsa.statespace.SARIMAX(data.CLOSE, order=(param[0], d, param[1]),
                                              seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
        aic = model.aic
        
        #Save best model, AIC and params
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([param, model.aic])
        
    result_tbl = pd.DataFrame(results)
    result_tbl.columns = ['params', 'aic']
    
    #Sort in ascending order, lower AIC is better
    result_tbl = result_tbl.sort_values(by='aic', ascending=True).reset_index(drop=True)
    return result_tbl

result_tbl = optimize_SARIMA(params_list, d, D, s)

#Set params that give the lowest AIC (Akaike Information Criteria)
p, q, P, Q = result_tbl.params[0]
best_model = sm.tsa.statespace.SARIMAX(data.CLOSE, order=(p, d, q), seasonal_order=(P, D, Q, s)).fit(disp=-1)
print(best_model.summary())

In [ ]:
#dataframe containing actual and predicted prices
comparison = pd.DataFrame({'actual': [18.93, 19.23, 19.08, 19.17, 19.11, 19.12],
                          'predicted': [18.96, 18.97, 18.96, 18.92, 18.94, 18.92]}, 
                          index = pd.date_range(start='2018-06-05', periods=6,))

#predicted vs actual
plt.figure(figsize=(15, 7))
plt.plot(comparison.actual)
plt.plot(comparison.predicted)
plt.title('Predicted closing price of New GF Inc (GF)')
plt.ylabel('Closing price ($)')
plt.xlabel('Trading day')
plt.legend(loc='best')
plt.grid(False)
plt.show()